In [2]:
import pandas as pd
import numpy as np
import warnings
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [3]:
def data_processing(scrimmage_file, team_total_yards, passing_yards):
  scrim = pd.read_csv(scrimmage_file)
  total = pd.read_csv(team_total_yards)
  passing = pd.read_csv(passing_yards)

  combined = pd.merge(scrim, total, on='Tm', how='left')
  combined = pd.merge(combined, passing, on='Player', how='left')

  combined['Player'] = combined['Player'].str.replace('[*+]', '', regex=True)
  combined['Ctch%'] = combined['Ctch%'].str.replace('[%]', '', regex=True)

  combined['Tm_x'] = combined['Tm_x'].fillna(combined['Tm_y'])
  combined['Tm_x'] = combined['Tm_x'].str.replace('NOR', 'NO', regex=True)
  combined['Tm_x'] = combined['Tm_x'].str.replace('LVR', 'LV', regex=True)
  combined['Tm_x'] = combined['Tm_x'].str.replace('KAN', 'KC', regex=True)
  combined['Tm_x'] = combined['Tm_x'].str.replace('TAM', 'TB', regex=True)
  combined['Tm_x'] = combined['Tm_x'].str.replace('GNB', 'GB', regex=True)
  combined['Tm_x'] = combined['Tm_x'].str.replace('NWE', 'NE', regex=True)
  combined['Tm_x'] = combined['Tm_x'].str.replace('SFO', 'SF', regex=True)



  combined['Age_x'] = combined['Age_x'].fillna(combined['Age_y'])
  combined['Pos_x'] = combined['Pos_x'].fillna(combined['Pos_y'])
  combined['G_x'] = combined['G_x'].fillna(combined['G_y'])
  combined['GS_x'] = combined['GS_x'].fillna(combined['GS_y'])
  combined['%Scrm'] = (combined['YScm'] / combined['Yds_y']) * 100

  combined.drop(['-9999', 'Rk_y', 'G_y', 'PF', 'Yds_y', 'Ply', 'Y/P', 'TO', 'FL', '1stD',
       'Cmp_x', 'Att_y', 'Yds.1_y', 'TD_y', 'Int_x', 'NY/A_x', '1stD.1',
       'Att.1', 'Yds.2', 'TD.1_y', 'Y/A_y', '1stD.2', 'Pen', 'Yds.3', '1stPy',
       'Sc%', 'TO%', 'EXP', 'Rk', 'Tm_y', 'Age_y', 'Pos_y', 'G', 'GS_y', 'Player-additional'], axis = 1, inplace = True)

  combined.rename(columns = {'Rk_x' : 'Rank', 'Tm_x' : 'Team', 'Age_x' : 'Age', 'Pos_x' : 'Pos', 'G_x' : 'Games Played', 'GS_x' : 'Games Started', 'Yds_x' : 'RecYds', 'Y/R' : 'YardsPerRec', 'TD_x' : 'RecTds', '1D_x' : 'RecFirstDowns', 'Succ%_x' : 'SuccessfulReception', 'Lng_x' : 'LongestCatch', 'R/G' : 'RecPerGame', 'Y/G_x' : 'RecYardsPerGame', 'Ctch%' : 'Catch%', 'Y/Tgt' : 'YardsPerTarget', 'Att_x' : 'RushAtt', 'Yds.1_x' : 'RushYds', 'TD.1_x' : 'RushTds', '1D.1' : 'RushFirstDowns', 'Succ%.1' : 'SuccessfulRush', 'Lng.1' : 'LongestRush', 'Y/A_x' : 'YardsPerRush', 'Y/G.1' : 'RushYardsPerGame', 'A/G' : 'RushAttemptsPerGame', 'Touch' : 'Touches', 'Y/Tch' : 'YardsPerTouch', 'YScm' : 'ScrimmageYards', 'RRTD' : 'ReceivingRushingTds', 'Fmb' : 'Fumbles', 'Cmp_y' : 'Completions', 'Att' : 'PassingAttempts', 'Cmp%' : 'CompletionPercentage', 'Yds' : 'PassingYards', 'TD' : 'PassingTds', 'TD%' : 'PassingTdPercentage', 'Int_y' : 'Interceptions', 'Int%' : 'InterceptionPercentge', '1D_y' : 'PassingFirstDowns', 'Succ%_y' : 'SuccessfulPass%', 'Lng_y' : 'LongestPass', 'Y/A' : 'YardsPerPassAttempt', 'AY/A' : 'AdjustedYardsPerPassAttempt', 'Y/C' : 'YardsPerCompletion', 'Y/G_y' : 'PassingYardsPerGame', 'Rate' : 'PasserRating', 'Sk' : 'Sacks', 'Yds.1' : 'SackYardsLost', 'Sk%' : 'Sacked%', 'NY/A_y' : 'NetYardsPerPassAttempt', 'ANY/A' : 'AdjustedNetYardsPerPassAttempt'}, inplace = True)

  features = ['Age', 'Games Played', 'Games Started',
       'Tgt', 'Rec', 'RecYds', 'YardsPerRec', 'RecTds', 'RecFirstDowns',
       'SuccessfulReception', 'LongestCatch', 'RecPerGame', 'RecYardsPerGame',
       'Catch%', 'YardsPerTarget', 'RushAtt', 'RushYds', 'RushTds',
       'RushFirstDowns', 'SuccessfulRush', 'LongestRush', 'YardsPerRush',
       'RushYardsPerGame', 'RushAttemptsPerGame', 'Touches', 'YardsPerTouch',
       'ScrimmageYards', 'ReceivingRushingTds', 'Fumbles', '%Scrm', 'Completions',
       'PassingAttempts', 'CompletionPercentage', 'PassingYards', 'PassingTds',
       'PassingTdPercentage', 'Interceptions', 'InterceptionPercentge',
       'PassingFirstDowns', 'SuccessfulPass%', 'LongestPass',
       'YardsPerPassAttempt', 'AdjustedYardsPerPassAttempt',
       'YardsPerCompletion', 'PassingYardsPerGame', 'PasserRating', 'QBR',
       'Sacks', 'SackYardsLost', 'Sacked%', 'NetYardsPerPassAttempt',
       'AdjustedNetYardsPerPassAttempt', '4QC', 'GWD']

  for i in features:
    combined[i] = pd.to_numeric(combined[i], errors = 'coerce')
    combined[f'{i}ToMean'] = combined[i] / combined[i].mean()
    combined = combined.drop(i, axis = 1)

  combined.fillna(0, inplace = True)

  positions = ['QB', 'RB', 'WR', 'TE']
  filtered_combine = combined[combined['Pos'].isin(positions)]

  manyTeams = ['2TM', '3TM', '4TM']
  filtered_combine = filtered_combine[~filtered_combine['Team'].isin(manyTeams)]

  filtered_combine['isQB'] = np.where(filtered_combine['Pos'] == 'QB', 1, 0)
  filtered_combine['isRB'] = np.where(filtered_combine['Pos'] == 'RB', 1, 0)
  filtered_combine['isWR'] = np.where(filtered_combine['Pos'] == 'WR', 1, 0)
  filtered_combine['isTE'] = np.where(filtered_combine['Pos'] == 'TE', 1, 0)


  return filtered_combine

In [4]:
season2023 = data_processing('/content/drive/MyDrive/RandomForestData/2023/2023Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2023/2023TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2023/2023Passing.csv')

season2023WinPercentage = {'BUF': 0.647, 'MIA': 0.647, 'NYJ': 0.412, 'NE': 0.235, 'BAL': 0.765, 'CLE': 0.647, 'PIT': 0.588, 'CIN': 0.529, 'HOU': 0.588, 'JAX': 0.529, 'IND': 0.529, 'TEN': 0.353, 'KC': 0.647, 'LV': 0.471, 'DEN': 0.471, 'LAC': 0.294, 'DAL': 0.706, 'PHI': 0.647, 'NYG': 0.353, 'WAS': 0.235, 'DET': 0.706, 'GB': 0.529, 'MIN': 0.412, 'CHI': 0.412, 'TB': 0.529, 'NO': 0.529, 'ATL': 0.412, 'CAR': 0.118, 'SF': 0.706, 'LAR': 0.588, 'SEA': 0.529, 'ARI': 0.235}
season2023MadePlayoffs = {'BUF': 1, 'MIA': 1, 'NYJ': 0, 'NE': 0, 'BAL': 1, 'CLE': 1, 'PIT': 1, 'CIN': 0, 'HOU': 1, 'JAX': 0, 'IND': 0, 'TEN': 0, 'KC': 1, 'LV': 0, 'DEN': 0, 'LAC': 0, 'DAL': 1, 'PHI': 1, 'NYG': 0, 'WAS': 0, 'DET': 1, 'GB': 1, 'MIN': 0, 'CHI': 0, 'TB': 1, 'NO': 0, 'ATL': 0, 'CAR': 0, 'SF': 1, 'LAR': 1, 'SEA': 0, 'ARI': 0}

season2023WinPercentageDF = pd.DataFrame(list(season2023WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2023MadePlayoffsDF = pd.DataFrame(list(season2023MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2023 = pd.merge(season2023, season2023WinPercentageDF, on = 'Team', how = 'left')
season2023 = pd.merge(season2023, season2023MadePlayoffsDF, on = 'Team', how = 'left')

season2023['wonMVP'] = np.where(season2023['Player'] == 'Lamar Jackson', 1, 0)
season2023['wonOPOY'] = np.where(season2023['Player'] == 'Christian McCaffrey', 1, 0)

In [5]:
season2022 = data_processing('/content/drive/MyDrive/RandomForestData/2022/2022Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2022/2022TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2022/2022Passing.csv')

season2022WinPercentage = {'KC': 0.824, 'PHI': 0.824, 'BUF': 0.813, 'MIN': 0.765, 'SF': 0.765, 'CIN': 0.75, 'DAL': 0.706, 'BAL': 0.588, 'LAC': 0.588, 'NYG': 0.559, 'DET': 0.529, 'JAX': 0.529, 'MIA': 0.529, 'PIT': 0.529, 'SEA': 0.529, 'TB': 0.471, 'GB': 0.471, 'NE': 0.471, 'ATL': 0.412, 'CAR': 0.412, 'CLE': 0.412, 'NO': 0.412, 'NYJ': 0.412, 'TEN': 0.412, 'ARI': 0.235, 'IND': 0.265, 'LAR': 0.294, 'DEN': 0.294, 'LV': 0.353, 'CHI': 0.176, 'HOU': 0.206, 'WAS': 0.5}
season2022MadePlayoffs = {'KC': 1, 'PHI': 1, 'BUF': 1, 'MIN': 1, 'SF': 1, 'CIN': 1, 'DAL': 1, 'BAL': 1, 'LAC': 1, 'NYG': 1, 'DET': 0, 'JAX': 1, 'MIA': 1, 'PIT': 0, 'SEA': 1, 'TB': 1, 'GB': 0, 'NE': 0, 'ATL': 0, 'CAR': 0, 'CLE': 0, 'NO': 0, 'NYJ': 0, 'TEN': 0, 'ARI': 0, 'IND': 0, 'LAR': 0, 'DEN': 0, 'LV': 0, 'CHI': 0, 'HOU': 0, 'WAS': 0}

season2022WinPercentageDF = pd.DataFrame(list(season2022WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2022MadePlayoffsDF = pd.DataFrame(list(season2022MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2022 = pd.merge(season2022, season2022WinPercentageDF, on = 'Team', how = 'left')
season2022 = pd.merge(season2022, season2022MadePlayoffsDF, on = 'Team', how = 'left')

season2022['wonMVP'] = np.where(season2022['Player'] == 'Patrick Mahomes', 1, 0)
season2022['wonOPOY'] = np.where(season2022['Player'] == 'Justin Jefferson', 1, 0)

In [6]:
season2021 = data_processing('/content/drive/MyDrive/RandomForestData/2021/2021Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2021/2021TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2021/2021Passing.csv')

season2021WinPercentage = {'GB': 0.765, 'TB': 0.765, 'DAL': 0.706, 'KC': 0.706, 'LAR': 0.706, 'TEN': 0.706, 'ARI': 0.647, 'BUF': 0.647, 'CIN': 0.588, 'LV': 0.588, 'NE': 0.588, 'SF': 0.588, 'PIT': 0.559, 'IND': 0.529, 'LAC': 0.529, 'MIA': 0.529, 'NO': 0.529, 'PHI': 0.529, 'MIN': 0.471, 'CLE': 0.471, 'BAL': 0.471, 'ATL': 0.412, 'DEN': 0.412, 'SEA': 0.412, 'WAS': 0.412, 'CHI': 0.353, 'CAR': 0.294, 'NYG': 0.235, 'DET': 0.206, 'NYJ': 0.206, 'JAX': 0.176, 'HOU': 0.176}
season2021MadePlayoffs = {'GB': 1, 'TB': 1, 'DAL': 1, 'KC': 1, 'LAR': 1, 'TEN': 1, 'ARI': 1, 'BUF': 1, 'CIN': 1, 'LV': 1, 'NE': 1, 'SF': 1, 'PIT': 1, 'IND': 0, 'LAC': 0, 'MIA': 0, 'NO': 0, 'PHI': 1, 'MIN': 0, 'CLE': 0, 'BAL': 0, 'ATL': 0, 'DEN': 0, 'SEA': 0, 'WAS': 0, 'CHI': 0, 'CAR': 0, 'NYG': 0, 'DET': 0, 'NYJ': 0, 'JAX': 0, 'HOU': 0}

season2021WinPercentageDF = pd.DataFrame(list(season2021WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2021MadePlayoffsDF = pd.DataFrame(list(season2021MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2021 = pd.merge(season2021, season2021WinPercentageDF, on = 'Team', how = 'left')
season2021 = pd.merge(season2021, season2021MadePlayoffsDF, on = 'Team', how = 'left')

season2021['wonMVP'] = np.where(season2021['Player'] == 'Aaron Rodgers', 1, 0)
season2021['wonOPOY'] = np.where(season2021['Player'] == 'Cooper Kupp', 1, 0)

In [7]:
season2020 = data_processing('/content/drive/MyDrive/RandomForestData/2020/2020Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2020/2020TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2020/2020Passing.csv')

season2020WinPercentage = {"KC": 0.875, "BUF": 0.813, "GB": 0.813, "NO": 0.75, "PIT": 0.75, "SEA": 0.75, "BAL": 0.688, "CLE": 0.688, "IND": 0.688, "TB": 0.688, "TEN": 0.688, "LAR": 0.625, "MIA": 0.625, "ARI": 0.5, "CHI": 0.5, "LV": 0.5, "LAC": 0.438, "MIN": 0.438, "NE": 0.438, "WAS": 0.438, "DAL": 0.375, "NYG": 0.375, "SF": 0.375, "CAR": 0.313, "DEN": 0.313, "DET": 0.313, "CIN": 0.281, "PHI": 0.281, "ATL": 0.25, "HOU": 0.25, "NYJ": 0.125, "JAX": 0.063}
season2020MadePlayoffs = {"KC": 1, "BUF": 1, "GB": 1, "NO": 1, "PIT": 1, "SEA": 1, "BAL": 1, "CLE": 1, "IND": 1, "TB": 1, "TEN": 1, "LAR": 1, "MIA": 0, "ARI": 0, "CHI": 1, "LV": 0, "LAC": 0, "MIN": 0, "NE": 0, "WAS": 1, "DAL": 0, "NYG": 0, "SF": 0, "CAR": 0, "DEN": 0, "DET": 0, "CIN": 0, "PHI": 0, "ATL": 0, "HOU": 0, "NYJ": 0, "JAX": 0}

season2020WinPercentageDF = pd.DataFrame(list(season2020WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2020MadePlayoffsDF = pd.DataFrame(list(season2020MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2020 = pd.merge(season2020, season2020WinPercentageDF, on = 'Team', how = 'left')
season2020 = pd.merge(season2020, season2020MadePlayoffsDF, on = 'Team', how = 'left')

season2020['wonMVP'] = np.where(season2020['Player'] == 'Aaron Rodgers', 1, 0)
season2020['wonOPOY'] = np.where(season2020['Player'] == 'Derrick Henry', 1, 0)

In [8]:
season2019 = data_processing('/content/drive/MyDrive/RandomForestData/2019/2019Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2019/2019TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2019/2019Passing.csv')

season2019WinPercentage = {"BAL": 0.875, "GB": 0.813, "NO": 0.813, "SF": 0.813, "KC": 0.75, "NE": 0.75, "SEA": 0.688, "BUF": 0.625, "HOU": 0.625, "MIN": 0.625, "PHI": 0.563, "TEN": 0.563, "LAR": 0.563, "PIT": 0.5, "CHI": 0.5, "DAL": 0.5, "ATL": 0.438, "DEN": 0.438, "IND": 0.438, "OAK": 0.438, "CLE": 0.375, "JAX": 0.375, "LAC": 0.375, "CAR": 0.313, "ARI": 0.313, "MIA": 0.313, "DET": 0.219, "NYG": 0.25, "WAS": 0.188, "CIN": 0.125, "NYJ": 0.438, "TB": 0.5}
season2019MadePlayoffs = {"BAL": 1, "GB": 1, "NO": 1, "SF": 1, "KC": 1, "NE": 1, "SEA": 1, "BUF": 1, "HOU": 1, "MIN": 1, "PHI": 1, "TEN": 1, "LAR": 0, "PIT": 0, "CHI": 0, "DAL": 0, "ATL": 0, "DEN": 0, "IND": 0, "OAK": 0, "CLE": 0, "JAX": 0, "LAC": 0, "CAR": 0, "ARI": 0, "MIA": 0, "DET": 0, "NYG": 0, "WAS": 0, "CIN": 0, "NYJ": 0, "TB": 0}

season2019WinPercentageDF = pd.DataFrame(list(season2019WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2019MadePlayoffsDF = pd.DataFrame(list(season2019MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2019 = pd.merge(season2019, season2019WinPercentageDF, on = 'Team', how = 'left')
season2019 = pd.merge(season2019, season2019MadePlayoffsDF, on = 'Team', how = 'left')

season2019['wonMVP'] = np.where(season2019['Player'] == 'Lamar Jackson', 1, 0)
season2019['wonOPOY'] = np.where(season2019['Player'] == 'Michael Thomas', 1, 0)

In [9]:
season2018 = data_processing('/content/drive/MyDrive/RandomForestData/2018/2018Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2018/2018TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2018/2018Passing.csv')

season2018WinPercentage = {"LAR": 0.813, "NO": 0.813, "CHI": 0.75, "KC": 0.75, "LAC": 0.75, "HOU": 0.688, "NE": 0.688, "BAL": 0.625, "DAL": 0.625, "IND": 0.625, "SEA": 0.625, "PIT": 0.594, "PHI": 0.563, "TEN": 0.563, "MIN": 0.531, "CLE": 0.469, "ATL": 0.438, "CAR": 0.438, "MIA": 0.438, "WAS": 0.438, "GB": 0.406, "BUF": 0.375, "CIN": 0.375, "DEN": 0.375, "DET": 0.375, "JAX": 0.313, "NYG": 0.313, "TB": 0.313, "OAK": 0.25, "NYJ": 0.25, "SF": 0.25, "ARI": 0.188}
season2018MadePlayoffs = {"LAR": 1, "NO": 1, "CHI": 1, "KC": 1, "LAC": 1, "HOU": 1, "NE": 1, "BAL": 1, "DAL": 1, "IND": 1, "SEA": 1, "PIT": 0, "PHI": 1, "TEN": 0, "MIN": 0, "CLE": 0, "ATL": 0, "CAR": 0, "MIA": 0, "WAS": 0, "GB": 0, "BUF": 0, "CIN": 0, "DEN": 0, "DET": 0, "JAX": 0, "NYG": 0, "TB": 0, "OAK": 0, "NYJ": 0, "SF": 0, "ARI": 0}

season2018WinPercentageDF = pd.DataFrame(list(season2018WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2018MadePlayoffsDF = pd.DataFrame(list(season2018MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2018 = pd.merge(season2018, season2018WinPercentageDF, on = 'Team', how = 'left')
season2018 = pd.merge(season2018, season2018MadePlayoffsDF, on = 'Team', how = 'left')

season2018['wonMVP'] = np.where(season2018['Player'] == 'Patrick Mahomes', 1, 0)
season2018['wonOPOY'] = np.where(season2018['Player'] == 'Patrick Mahomes', 1, 0)

In [10]:
season2017 = data_processing('/content/drive/MyDrive/RandomForestData/2017/2017Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2017/2017TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2017/2017Passing.csv')

season2017WinPercentage = {"MIN": 0.813, "NE": 0.813, "PHI": 0.813, "PIT": 0.813, "CAR": 0.688, "LAR": 0.688, "NO": 0.688, "ATL": 0.625, "JAX": 0.625, "KC": 0.625, "BAL": 0.563, "BUF": 0.563, "DAL": 0.563, "DET": 0.563, "LAC": 0.563, "SEA": 0.563, "TEN": 0.563, "ARI": 0.5, "CIN": 0.438, "GB": 0.438, "WAS": 0.438, "OAK": 0.375, "MIA": 0.375, "SF": 0.375, "CHI": 0.313, "DEN": 0.313, "NYJ": 0.313, "TB": 0.313, "HOU": 0.25, "IND": 0.25, "NYG": 0.188, "CLE": 0.0}
season2017MadePlayoffs = {"MIN": 1, "NE": 1, "PHI": 1, "PIT": 1, "CAR": 1, "LAR": 1, "NO": 1, "ATL": 1, "JAX": 1, "KC": 1, "BAL": 0, "BUF": 1, "DAL": 0, "DET": 0, "LAC": 0, "SEA": 0, "TEN": 1, "ARI": 0, "CIN": 0, "GB": 0, "WAS": 0, "OAK": 0, "MIA": 0, "SF": 0, "CHI": 0, "DEN": 0, "NYJ": 0, "HOU" : 0, "IND" : 0, "CLE" : 0, "NYG" : 0, "TB" : 0}

season2017WinPercentageDF = pd.DataFrame(list(season2017WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2017MadePlayoffsDF = pd.DataFrame(list(season2017MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2017 = pd.merge(season2017, season2017WinPercentageDF, on = 'Team', how = 'left')
season2017 = pd.merge(season2017, season2017MadePlayoffsDF, on = 'Team', how = 'left')

season2017['wonMVP'] = np.where(season2017['Player'] == 'Tom Brady', 1, 0)
season2017['wonOPOY'] = np.where(season2017['Player'] == 'Todd Gurley', 1, 0)

In [11]:
season2016 = data_processing('/content/drive/MyDrive/RandomForestData/2016/2016Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2016/2016TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2016/2016Passing.csv')

season2016WinPercentage = {"NE": 0.875, "MIA": 0.625, "BUF": 0.438, "NYJ": 0.313, "PIT": 0.688, "BAL": 0.5, "CIN": 0.406, "CLE": 0.063, "HOU": 0.563, "TEN": 0.563, "IND": 0.5, "JAX": 0.188, "KC": 0.75, "OAK": 0.75, "DEN": 0.563, "SDG": 0.313, "DAL": 0.813, "NYG": 0.688, "WAS": 0.531, "PHI": 0.438, "GB": 0.625, "DET": 0.563, "MIN": 0.5, "CHI": 0.188, "ATL": 0.688, "TB": 0.563, "NO": 0.438, "CAR": 0.375, "SEA": 0.656, "ARI": 0.469, "LAR": 0.25, "SF": 0.125}
season2016MadePlayoffs = {"NE": 1, "MIA": 1, "BUF": 0, "NYJ": 0, "PIT": 1, "BAL": 0, "CIN": 0, "CLE": 0, "HOU": 1, "TEN": 0, "IND": 0, "JAX": 0, "KC": 1, "OAK": 1, "DEN": 0, "SDG": 0, "DAL": 1, "NYG": 1, "WAS": 0, "PHI": 0, "GB": 1, "DET": 1, "MIN": 0, "CHI": 0, "ATL": 1, "TB": 0, "NO": 0, "CAR": 0, "SEA": 1, "ARI": 0, "LAR": 0, "SF": 0}

season2016WinPercentageDF = pd.DataFrame(list(season2016WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2016MadePlayoffsDF = pd.DataFrame(list(season2016MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2016 = pd.merge(season2016, season2016WinPercentageDF, on = 'Team', how = 'left')
season2016 = pd.merge(season2016, season2016MadePlayoffsDF, on = 'Team', how = 'left')

season2016['wonMVP'] = np.where(season2016['Player'] == 'Matt Ryan', 1, 0)
season2016['wonOPOY'] = np.where(season2016['Player'] == 'Matt Ryan', 1, 0)

In [12]:
season2015 = data_processing('/content/drive/MyDrive/RandomForestData/2015/2015Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2015/2015TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2015/2015Passing.csv')

season2015WinPercentage = {"CAR": 0.938, "ARI": 0.813, "CIN": 0.75, "DEN": 0.75, "NE": 0.75, "KC": 0.688, "MIN": 0.688, "GB": 0.625, "NYJ": 0.625, "PIT": 0.625, "SEA": 0.625, "HOU": 0.563, "WAS": 0.563, "ATL": 0.5, "BUF": 0.5, "IND": 0.5, "DET": 0.438, "OAK": 0.438, "STL": 0.438, "NO": 0.438, "PHI": 0.438, "CHI": 0.375, "MIA": 0.375, "TB": 0.375, "BAL": 0.313, "JAX": 0.313, "SF": 0.313, "DAL": 0.25, "SDG": 0.25, "CLE": 0.188, "TEN": 0.188, "NYG": 0.375}
season2015MadePlayoffs = {"CAR": 1, "ARI": 1, "CIN": 1, "DEN": 1, "NE": 1, "KC": 1, "MIN": 1, "GB": 1, "NYJ": 0, "PIT": 1, "SEA": 1, "HOU": 1, "WAS": 1, "ATL": 0, "BUF": 0, "IND": 0, "DET": 0, "OAK": 0, "STL": 0, "NO": 0, "PHI": 0, "CHI": 0, "MIA": 0, "TB": 0, "BAL": 0, "JAX": 0, "SF": 0, "DAL": 0, "SDG": 0, "CLE": 0, "TEN": 0, "NYG": 0}

season2015WinPercentageDF = pd.DataFrame(list(season2015WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2015MadePlayoffsDF = pd.DataFrame(list(season2015MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2015 = pd.merge(season2015, season2015WinPercentageDF, on = 'Team', how = 'left')
season2015 = pd.merge(season2015, season2015MadePlayoffsDF, on = 'Team', how = 'left')

season2015['wonMVP'] = np.where(season2015['Player'] == 'Cam Newton', 1, 0)
season2015['wonOPOY'] = np.where(season2015['Player'] == 'Cam Newton', 1, 0)

In [13]:
season2014 = data_processing('/content/drive/MyDrive/RandomForestData/2014/2014Scrimmage.csv', '/content/drive/MyDrive/RandomForestData/2014/2014TotalYards.csv', '/content/drive/MyDrive/RandomForestData/2014/2014Passing.csv')

season2014WinPercentage = {"DAL": 0.75, "DEN": 0.75, "GB": 0.75, "NE": 0.75, "SEA": 0.75, "ARI": 0.688, "DET": 0.688, "IND": 0.688, "PIT": 0.688, "CIN": 0.656, "BAL": 0.625, "PHI": 0.625, "BUF": 0.563, "HOU": 0.563, "KC": 0.563, "SDG": 0.563, "MIA": 0.5, "SF": 0.5, "CAR": 0.469, "CLE": 0.438, "MIN": 0.438, "NO": 0.438, "ATL": 0.375, "STL": 0.375, "NYG": 0.375, "CHI": 0.313, "NYJ": 0.25, "WAS": 0.25, "JAX": 0.188, "OAK": 0.188, "TB": 0.125, "TEN": 0.125}
season2014MadePlayoffs = {"DAL": 1, "DEN": 1, "GB": 1, "NE": 1, "SEA": 1, "ARI": 1, "DET": 1, "IND": 1, "PIT": 1, "CIN": 1, "BAL": 1, "PHI": 0, "BUF": 0, "HOU": 0, "KC": 0, "SDG": 0, "MIA": 0, "SF": 0, "CAR": 1, "CLE": 0, "MIN": 0, "NO": 0, "ATL": 0, "STL": 0, "NYG": 0, "CHI": 0, "NYJ": 0, "WAS": 0, "JAX": 0, "OAK": 0, "TB": 0, "TEN": 0}

season2014WinPercentageDF = pd.DataFrame(list(season2014WinPercentage.items()), columns = ['Team', 'Win Percentage'])
season2014MadePlayoffsDF = pd.DataFrame(list(season2014MadePlayoffs.items()), columns = ['Team', 'madePlayoffs'])

season2014 = pd.merge(season2014, season2014WinPercentageDF, on = 'Team', how = 'left')
season2014 = pd.merge(season2014, season2014MadePlayoffsDF, on = 'Team', how = 'left')

season2014['wonMVP'] = np.where(season2014['Player'] == 'Aaron Rodgers', 1, 0)
season2014['wonOPOY'] = np.where(season2014['Player'] == 'DeMarco Murray', 1, 0)

In [22]:
training_dataframes = [season2014, season2015, season2016, season2017, season2018, season2019, season2020, season2021, season2022]
training_data = pd.concat(training_dataframes)

In [23]:
test_data  = pd.DataFrame(season2023)

In [24]:
columns_to_drop = ['Rank', 'Player', 'Team', 'Pos']

X_train = training_data.drop(columns_to_drop + ['wonOPOY'], axis = 1)
y_train = training_data['wonOPOY']

X_test = test_data.drop(columns_to_drop + ['wonOPOY'], axis=1)
y_test = test_data['wonOPOY']

player_info = season2023['Player']

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [29]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)

grid_search.fit(X_train_smote, y_train_smote)

print("Best parameters found: ", grid_search.best_params_)
print("Best average ROC AUC found: ", grid_search.best_score_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best parameters found:  {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.7}
Best average ROC AUC found:  0.9999992714520932


In [35]:
model = xgb.XGBClassifier(
    colsample_bytree=1.0,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=300,
    subsample=0.7,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train_smote, y_train_smote)

predictions = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

Accuracy: 0.9960159362549801
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       501
           1       0.00      0.00      0.00         1

    accuracy                           1.00       502
   macro avg       0.50      0.50      0.50       502
weighted avg       1.00      1.00      1.00       502



In [17]:
cm = confusion_matrix(y_test, predictions)
cm

array([[500,   1],
       [  0,   1]])

In [ ]:
results_df = pd.DataFrame({
    'Player': player_info,
    'Actual': y_test,
    'Predicted': predictions
})
print(results_df[results_df['Predicted'] == 1])

In [34]:
scores = cross_val_score(model, X_train_smote, y_train_smote, cv=5)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [0.99779614 0.99834711 0.99834711 0.99724518 0.99779493]
Mean cross-validation score: 0.9979060931050509
